In [1]:
import duckdb

In [3]:
con = duckdb.connect(database=':memory:')
con.sql('INSTALL spatial; LOAD spatial')

In [10]:
con.sql('''
CREATE OR REPLACE VIEW courts AS
SELECT
    * EXCLUDE (geometry),
    ST_GeomFromWKB(geometry) AS GEOMETRY
FROM
    '../Courts.parquet';
SELECT * FROM courts LIMIT 5;
''')

┌──────────┬──────────────────────┬─────────────────┬───┬──────────────┬──────────────┬──────────────────────┐
│ OBJECTID │       GLOBALID       │    ASSET_ID     │ … │ CREATED_USER │ UPDATED_USER │       GEOMETRY       │
│  int64   │       varchar        │     varchar     │   │   varchar    │   varchar    │       geometry       │
├──────────┼──────────────────────┼─────────────────┼───┼──────────────┼──────────────┼──────────────────────┤
│      125 │ {EF8386EA-B072-4EF…  │ MC-D54-COURT-02 │ … │ NULL         │ parksparks   │ MULTIPOLYGON (((13…  │
│      124 │ {70B2E5F8-7C72-4A9…  │ MC-D54-COURT-01 │ … │ NULL         │ parksparks   │ MULTIPOLYGON (((13…  │
│      545 │ {9A9E1645-D545-4E6…  │ MC-D47-COURT-01 │ … │ NULL         │ PARKS        │ MULTIPOLYGON (((12…  │
│      546 │ {F5DF1C67-11B3-4BD…  │ MC-D47-COURT-04 │ … │ NULL         │ PARKS        │ MULTIPOLYGON (((12…  │
│      547 │ {5A6DC082-71DE-46A…  │ MC-D47-COURT-05 │ … │ NULL         │ PARKS        │ MULTIPOLYGON (((12…  │
├

In [26]:
con.sql('''
CREATE OR REPLACE VIEW parks AS
SELECT
    * EXCLUDE (geometry),
    ST_GeomFromWKB(geometry) AS GEOMETRY
FROM
    '../ParkUnits.parquet';
SELECT * FROM parks LIMIT 5;
''')

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name  │ column_type │  null   │   key   │ default │  extra  │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ OBJECTID      │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ PARK_NAME     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ OWNER         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ STATUS        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ FACILITY_C    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ TYPE_         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ AUTHORITY     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ APPROVAL_YEAR │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ PLANNING_AREA │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ACQ_BASIS     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NU

In [33]:
con.sql('''
SELECT
    column_name
FROM
    (DESCRIBE courts)
WHERE
    column_type LIKE 'INT%'
''')

┌──────────────┐
│ column_name  │
│   varchar    │
├──────────────┤
│ BASKETBALL   │
│ BOCCI        │
│ FUTSAL       │
│ HANDBALL     │
│ HOCKEY       │
│ PADDLEBALL   │
│ PICKLEBALL   │
│ SHUFFLEBOARD │
│ RACQUETBALL  │
│ TAICHI       │
│ TENNIS       │
│ VOLLEYBALL   │
│ LIGHTS       │
│ PERMIT       │
│ FULLHALF     │
│ STANDARDS    │
├──────────────┤
│   16 rows    │
└──────────────┘

In [37]:
con.sql('''
CREATE OR REPLACE VIEW parks_enriched AS
SELECT
    parks.*,
    COUNT(courts.*) AS courts_count,
    COALESCE(BASKETBALL.SUM(), 0) AS basketball_count,
    COALESCE(BOCCI.SUM(), 0) AS bocci_count,
    COALESCE(FUTSAL.SUM(), 0) AS futsal_count,
    COALESCE(HANDBALL.SUM(), 0) AS handball_count,
    COALESCE(HOCKEY.SUM(), 0) AS hockey_count,
    COALESCE(PADDLEBALL.SUM(), 0) AS paddleball_count,
    COALESCE(PICKLEBALL.SUM(), 0) AS pickleball_count,
    COALESCE(SHUFFLEBOARD.SUM(), 0) AS shuffleboard_count,
    COALESCE(RACQUETBALL.SUM(), 0) AS racquetball_count,
    COALESCE(TAICHI.SUM(), 0) AS taichi_count,
    COALESCE(TENNIS.SUM(), 0) AS tennis_count,
    COALESCE(VOLLEYBALL.SUM(), 0) AS volleyball_count
FROM
    parks
LEFT JOIN courts ON
    ST_Contains(parks.geometry, courts.geometry)
WHERE
    parks.owner = 'M-NCPPC'
    AND parks.status = 'Existing'
GROUP BY parks.*
ORDER BY courts_count DESC;
SELECT * FROM parks_enriched LIMIT 5;
''')

┌──────────┬──────────────────────┬─────────┬───┬───────────────────┬──────────────┬──────────────┬──────────────────┐
│ OBJECTID │      PARK_NAME       │  OWNER  │ … │ racquetball_count │ taichi_count │ tennis_count │ volleyball_count │
│  int64   │       varchar        │ varchar │   │      int128       │    int128    │    int128    │      int128      │
├──────────┼──────────────────────┼─────────┼───┼───────────────────┼──────────────┼──────────────┼──────────────────┤
│      218 │ Olney Manor Recrea…  │ M-NCPPC │ … │                 1 │            0 │           18 │                2 │
│      731 │ Cabin John Regiona…  │ M-NCPPC │ … │                 0 │            1 │           15 │                2 │
│      853 │ Wheaton Regional P…  │ M-NCPPC │ … │                 0 │            0 │           12 │                0 │
│       27 │ Fairland Recreatio…  │ M-NCPPC │ … │                 0 │            0 │            6 │                0 │
│      408 │ Norwood Local Park   │ M-NCPPC │ … 

In [38]:
con.sql('''
COPY (SELECT * FROM parks_enriched) TO '../parks_enriched.parquet' (FORMAT PARQUET);
''')